In [1]:
import pandas as pd

In [2]:
isin = pd.read_parquet('Z:/dataset/Refinitiv/common/permisindata')
isin.shape
isin.head()

(150607209, 4)

,InstrPermID,StartDate,EndDate,ISIN
0,281281,20170401,99991231,US92837G1004
1,281281,19940827,20120626,US9898341063
2,281281,20120626,20170401,US9898342053
3,4027246,17530101,20200411,JP360540AL40
4,4027247,17530101,20200604,JP360540AL65


In [3]:
ric = pd.read_parquet('Z:/dataset/Refinitiv/common/permricdata')
ric.shape
ric.head()

(601726160, 4)

,QuotePermID,StartDate,EndDate,RIC
0,36808,17530101,20200320,US30DCP11=PYNY
1,36821,17530101,20200320,US60DCP11=PYNY
2,36827,17530101,20200320,US90DCP11=PYNY
3,57249,17530101,20200320,US15DCP11=PYNY
4,57251,17530101,20200320,US180DCP11=PYNY


In [ ]:
m = pd.merge(isin, ric, on='PermID', how='inner')

In [4]:
ric.loc[ric['QuotePermID'] == 281281]

,QuotePermID,StartDate,EndDate,RIC


In [5]:
quote = pd.read_parquet('Z:/dataset/Refinitiv/common/permquoteinfo')
quote.shape
quote.head()

(310885589, 8)

,QuotePermID,Category,ShortName,RSCAssetCla,Opol,TrdWhenIssued,Is144aReg,RetireDate
0,21474836502,ORD,O-NET COMM GP,ORD,None,0,0,20201020.0
1,21474836541,RTS,IMAGI INT'L RTS,RTS,None,0,0,20100503.0
2,21474836542,RTS,IMAGI INT'L RTS,RTS,None,0,0,20100503.0
3,21474836561,DRC,ASHCO NIU GDR,GDR,None,0,0,20140403.0
4,21474836566,ORD,ECOLAB,ORD,XBER,0,0,NaN


In [6]:
quote_ref = pd.read_parquet('Z:/dataset/Refinitiv/common/permquoteref')
quote_ref.shape
quote_ref.head()

(201212786, 13)

,QuotePermID,Status,TypeCode,IsPrimary,Mic,Ticker,ComName,ExchCode,CurrPermID,InstrPermID,PriEstPermID,LotSize,DsQuoteNumber
0,21474836502,DC,EQUITY,1.0,None,None,O-NET TECH GP ORD,HKG,500177.0,2.147496e+10,3.006486e+10,NaN,69229X
1,21474836541,DC,EQUITY,1.0,None,2903,IMAGI INT'L RTS,HKG,500177.0,2.147484e+10,3.006478e+10,NaN,None
2,21474836542,DC,EQUITY,0.0,None,None,IMAGI INT'L RTS,HKG,500177.0,2.147484e+10,3.006478e+10,NaN,None
3,21474836561,DC,EQUITY,1.0,None,000161722,ASHCO NIULAB INDUSTRIES GDR,LUX,500110.0,2.147484e+10,NaN,NaN,69215W
4,21474836566,AC,EQUITY,0.0,XXXX,ECJ,ECOLAB ORD,DEU,500111.0,8.590938e+09,3.006484e+10,NaN,282643


In [7]:
quote_ref = quote_ref[['QuotePermID', 'InstrPermID']].drop_duplicates()
quote_ref.shape
quote_ref.head()

(201212786, 2)

,QuotePermID,InstrPermID
0,21474836502,2.147496e+10
1,21474836541,2.147484e+10
2,21474836542,2.147484e+10
3,21474836561,2.147484e+10
4,21474836566,8.590938e+09


In [8]:
ric2 = pd.merge(ric, quote_ref, on='QuotePermID', how='inner')
ric2.shape
ric2.head()

(397122252, 5)

,QuotePermID,StartDate,EndDate,RIC,InstrPermID
0,21474836502,20100419,20201019,0877.HK,2.147496e+10
1,21474836502,20201019,99991231,0877.HK^J20,2.147496e+10
2,21474836541,20100419,20100503,2903.HK,2.147484e+10
3,21474836541,20100503,99991231,2903.HK^E10,2.147484e+10
4,21474836542,20100419,20100503,2903ta.HK,2.147484e+10


In [9]:
ric3 = pd.merge(ric2, isin, on='InstrPermID', how='inner')
ric3.shape
ric3.head()

(215775393, 8)

,QuotePermID,StartDate_x,EndDate_x,RIC,InstrPermID,StartDate_y,EndDate_y,ISIN
0,21474836502,20100419,20201019,0877.HK,2.147496e+10,20100428,99991231,KYG6771C1006
1,21474836502,20201019,99991231,0877.HK^J20,2.147496e+10,20100428,99991231,KYG6771C1006
2,21474836541,20100419,20100503,2903.HK,2.147484e+10,20100420,99991231,BMG476291452
3,21474836541,20100503,99991231,2903.HK^E10,2.147484e+10,20100420,99991231,BMG476291452
4,21474836542,20100419,20100503,2903ta.HK,2.147484e+10,20100420,99991231,BMG476291452


In [10]:
ric3['StartDate'] = ric3[['StartDate_x', 'StartDate_y']].max(axis=1)
ric3['EndDate'] = ric3[['EndDate_x', 'EndDate_y']].min(axis=1)

ric4 = ric3.loc[ric3['StartDate'] <= ric3['EndDate']]
ric4.shape
ric4.head()

(206599578, 10)

,QuotePermID,StartDate_x,EndDate_x,RIC,InstrPermID,StartDate_y,EndDate_y,ISIN,StartDate,EndDate
0,21474836502,20100419,20201019,0877.HK,2.147496e+10,20100428,99991231,KYG6771C1006,20100428,20201019
1,21474836502,20201019,99991231,0877.HK^J20,2.147496e+10,20100428,99991231,KYG6771C1006,20201019,99991231
2,21474836541,20100419,20100503,2903.HK,2.147484e+10,20100420,99991231,BMG476291452,20100420,20100503
3,21474836541,20100503,99991231,2903.HK^E10,2.147484e+10,20100420,99991231,BMG476291452,20100503,99991231
4,21474836542,20100419,20100503,2903ta.HK,2.147484e+10,20100420,99991231,BMG476291452,20100420,20100503


In [ ]:
ric4 = ric4[['RIC', 'ISIN', 'StartDate', 'EndDate']].drop_duplicates()
ric4.shape
ric4.head()

(206541481, 4)

,RIC,ISIN,StartDate,EndDate
0,0877.HK,KYG6771C1006,20100428,20201019
1,0877.HK^J20,KYG6771C1006,20201019,99991231
2,2903.HK,BMG476291452,20100420,20100503
3,2903.HK^E10,BMG476291452,20100503,99991231
4,2903ta.HK,BMG476291452,20100420,20100503


In [15]:
# ric4.to_parquet('../isin/ric_full.parquet', index=False)

In [13]:
ric5 = ric4[['ISIN', 'RIC']].drop_duplicates()
ric5.shape
ric5.head()

(203841972, 2)

,ISIN,RIC
0,KYG6771C1006,0877.HK
1,KYG6771C1006,0877.HK^J20
2,BMG476291452,2903.HK
3,BMG476291452,2903.HK^E10
4,BMG476291452,2903ta.HK


In [14]:
ric5.to_parquet('../isin/ric.parquet', index=False)